## <center> 3D Mesh loaded by PlyIO and plotted with triangle edges </center>

In [ ]:
using PlyIO
using PlotlyJS
using Rotations, StaticArrays
include("src/misc.jl");

In [ ]:
ply = load_ply("data/beethoven.ply")
x = ply["vertex"]["x"]
y = ply["vertex"]["y"]
z = ply["vertex"]["z"]
triangles = ply["face"]["vertex_indices"];

In [ ]:
#apply RotZ(angle2) * RotX(angle1) to each vertex of the triangulation, 
# to get Beethoven's vertical position 

myRot = RotZ(4*pi/9+pi/4)*RotX(pi/4)  
verts_matr = hcat(x, y, z)'
verts_matr = (myRot* verts_matr)' 
typeof(triangles)

ListProperty definition:
```
mutable struct ListProperty{S,T} <: AbstractVector{Vector{T}}
    name::String
    start_inds::Vector{Int}
    data::Vector{T}
end
```

In [ ]:
tri_matr = reshape(triangles.data, (3, length(triangles)))';

pl_mesh = PlotlyJS.mesh3d(x=verts_matr[:, 1],
                 y= verts_matr[ :, 2],
                 z=verts_matr[:, 3],
                 colorscale=[[0.0, "rgb(153, 153, 153)"], [1.0, "rgb(255,255,255)"]], 
                 intensity= verts_matr[:, 3],
                 flatshading=true,
                 i=tri_matr[:, 1],
                 j=tri_matr[:, 2],
                 k= tri_matr[:, 3],
                 name="Beethoven",
                 showscale=false);

pl_mesh["cmin"] = -7 # trick to ensure that B's face is not too dark
pl_mesh["lighting"] = attr(ambient=0.18,
                           diffuse=1,
                           fresnel=0.1,
                           specular=1,
                           roughness=0.05,
                           facenormalsepsilon=1e-15,
                           vertexnormalsepsilon=1e-15)
pl_mesh["lightposition"] = attr(x=100,
                                y=200,
                                z=0);                      

layout = Layout(
             title="Beethoven<br>Mesh3d with flatshading",
             font=attr(size=16, color="white"),
             width=600,
             height=600,
             scene_xaxis_visible=false,
             scene_yaxis_visible=false,
             scene_zaxis_visible=false,
             scene_camera_eye=attr(x=1.25, y=1.25, z=0.75),
             paper_bgcolor="rgb(50,50,50)")

pl = Plot([pl_mesh, tri_edges(verts_matr, tri_matr)], layout)

![Beethoven](images/Beethoven-ply.png)